In [16]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import os
import pandas as pd
from datetime import datetime
import time

from riotwatcher import LolWatcher, ApiError

""" 
API Rate Limits:
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
"""

watcher = LolWatcher(config['riotwatcher_api_key'])

In [17]:
# record start time
start_timer = time.time()

""" user-defined variables """
var_region = "euw1"
var_summoner_name= "AdAstraData"

""" fetch item_list from data_dragon and create dict with item_id : item_name """
data_dragon_versions = watcher.data_dragon.versions_for_region(region=var_region)
data_dragon_item_version = data_dragon_versions['n']['item']

item_list = watcher.data_dragon.items(data_dragon_item_version)

dict_item_id_item_name = {}

for item_id in item_list['data'].keys():
    df_item_id = pd.json_normalize(data=item_list['data'][item_id])
    item_name = df_item_id['name'].to_string(index=False)
    dict_item_id_item_name[item_id] = item_name

""" store summoner ids """
summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_summoner_name)
my_puuid = summoner_puuid['puuid']

""" find all matches played by puuid """
last_matches_puuid = watcher.match.matchlist_by_puuid(
    region=var_region, 
    puuid=my_puuid, 
    count=100
    )

# record end time
end_timer = time.time()

# number of API calls
num_api_calls = 4

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 2.568451404571533 s
Number of API calls: 4
API calls per second: 1.56


In [27]:
# import sqlite3
# from sqlalchemy import create_engine

# engine = create_engine('sqlite:///sqlite_test.db', echo=False)

# df = pd.DataFrame([[1,52],[1,2342]], columns=['a', 'b'])
# print(df)

# df.to_sql('mytable', con=engine, if_exists='append')

# df = pd.read_sql_query("SELECT * from mytable", con=engine)
# df

In [7]:
""" list of match_id for all matches played by summoner"""
len(last_matches_puuid)

100

In [4]:
# record start time
start_timer = time.time()

list_match_item_purchased = []

""" for each match_id """
for var_match_id in last_matches_puuid[0:26]:

    """ store participant's puuids """
    dict_participants_puuids = {}

    match_participants = watcher.match.by_id(region=var_region, match_id=var_match_id)
    list_match_participants = match_participants['metadata']['participants']
    
    for idx in range(len(list_match_participants)):
        participant_id = idx + 1
        dict_participants_puuids[str(participant_id)] = list_match_participants[idx]

    """ fetch match_timeline """
    match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)
    match_timeline = pd.json_normalize(data=match_info['info']['frames'], record_path='events')

    """ add match_id to match_timeline"""
    match_timeline['match_id'] = var_match_id

    """ convert match_timeline' timestamp to elapsed minutes"""
    match_timeline['timestamp_timedelta'] = pd.to_timedelta(match_timeline['timestamp'], unit='ms')
    match_timeline['elapsed_minutes'] = round(match_timeline['timestamp_timedelta']/ pd.Timedelta(minutes=1),3)
    
    """ subset for item_purchased """
    item_purchased = match_timeline.query("type == 'ITEM_PURCHASED'").dropna(axis=1)

    """ retrieve item_name """
    item_purchased['item_id'] = item_purchased['itemId'].astype(int)
    item_purchased['item_name'] = [dict_item_id_item_name[str(item)] for item in item_purchased['item_id']]

    """ retrieve summoner_puuid """
    item_purchased['participant_id'] = item_purchased['participantId'].astype(int)
    item_purchased['summoner_puuid'] = [dict_participants_puuids[str(id)] for id in item_purchased['participant_id']]

    """ subset for @my_puuid """
    item_purchased = item_purchased.query("summoner_puuid == @my_puuid")

    """ select cols - elapsed_minutes, item_name, summoner_puuid """
    item_purchased = item_purchased[
        [
            'match_id', 
            'elapsed_minutes',
            'item_name',
            'summoner_puuid'
        ]
    ]
    
    """ create dict with item_purchased per timestamp per participant (summoner) """
    dict_item_purchased = item_purchased.to_dict(orient='records')
    list_match_item_purchased.append(dict_item_purchased)

# record end time
end_timer = time.time()

# number of API calls
num_api_calls_per_cycle = 1
num_api_cycles = len(last_matches_puuid)
num_api_calls = num_api_cycles * num_api_calls_per_cycle

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 14.254740476608276 s
Number of API calls: 100
API calls per second: 7.02


In [5]:
""" obtain avg. elapsed_minutes per item_name (last matches)"""
from itertools import chain

df_match_item_purchased = pd.DataFrame(list(chain.from_iterable(list_match_item_purchased)))

avg_min_Dorans_Ring = df_match_item_purchased.query('item_name == "Doran\'s Ring"')['elapsed_minutes'].describe()
avg_min_Dorans_Ring

count    24.000000
mean      0.259708
std       0.146596
min       0.097000
25%       0.155500
50%       0.209000
75%       0.322000
max       0.614000
Name: elapsed_minutes, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 1. need to test first for API call before proceeding (hitting the API's rate limits as it is) 
# 2. need to add order_num to item_name to distinguish between similar item_names bought at different timestamps
# 3. .describe() to be applied per item_name_order_num 